### json 불러와서 저장하기

In [1]:
import numpy as np
import pandas as pd
import json
import jpype
import glob
from random import shuffle
from bs4 import BeautifulSoup as bs

data_path = "data"
#data_path = "data(test)"

file_list = glob.glob("%s/*.json" % data_path)
shuffle(file_list)

json_train=[]
for json_file_name in file_list:
    print json_file_name
    json_file = json.loads(open(json_file_name).read())
    json_train += json_file["articles"]

data/3.json
data/15.json
data/14.json
data/10.json
data/2.json
data/11.json
data/4.json
data/1.json
data/17.json
data/8.json
data/13.json
data/7.json
data/9.json
data/5.json
data/6.json
data/16.json
data/12.json
data/0.json


### json에서 text 형태소 추출

In [2]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    text = bs(article["text"],"lxml").text
    
    if cnt % 10000 == 0:
        print cnt

    labeled_train.append({
            "istroll": article["is_troll"],
            "text_pos": ["%s_%s" % (first, second) for first, second in mecab.pos(text)],
            "pk": article["pk"]
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train = labeled_train.set_index('pk')

labeled_train

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.ilbe.com/2691808050" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://withgonggam.tistory.com/m/post/view/id/1316" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


0
10000
20000
30000
40000

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://m.khan.co.kr/view.html?category=2&med_id=khan&artid=201401071905352&code=361102" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://news.sportsseoul.com/read/entertain/1295201.htm<div><span><br><br>하아...</span></div>" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and 


50000
60000

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://ilwar.com/free/44658" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://ilwar.com/free/45344" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http:

...


130000
140000
150000
160000

/Library/Python/2.7/site-packages/bs4/__init__.py:198: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://blog.naver.com/kakashi08/110086751501" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://ilwar.com/free/5920
http://ilwar.com/free/6078" looks like a URL. Beautiful Soup is not an HTTP client. You should probabl


170000

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://m.bgmstore.net/view/049Ju" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://m.news.naver.com/read.nhn?sid1=100&oid=055&aid=0000269444" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


,istroll,text_pos
pk,,
32102,False,"[고생_NNG, 이_JKS, 많_VA, 농_NNG, ._SY, ㅋㅋ_UNKNOWN,..."
32103,False,"[네이버_NNP, 아이디_NNG, mchancho_SL, 대구_NNP, 수성구_NN..."
32104,False,"[반대_NNG, 누르_VV, 고_EC, 다닌다고_VV+EC, 손가락_NNG, 아프_..."
32105,False,"[왜_MAG, 베_VV+EC, 츙이들이_UNKNOWN, 여기_NP, 와서_VV+EC..."
32106,False,"[여긴_NP+JX, 베_NNG, 충_NNG, 들_XSN, 쓰_VV, 는_ETM, 욕..."
32107,False,"[박멸_NNG, 농장_NNG, 맛보_VV, 러_EC, 왔_VX+EP, 습니다_EF]"
32108,False,"[3_SN, 대_NNBC, 가_JKS, 일_NR, 베_NNG, 충_NNG, ㅋㅋㅋ_..."
32109,True,"[1_SN, ._SF, 이명박_NNP, 임기_NNG, 초기_NNG, 에_JKB, 광..."
32110,False,"[가입_NNG, 인사_NNG, 드립니다_VV+EF, ._SF, 앞_NNG, 으로_J..."


###bag of words

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

def make_bag_of_words(labeled_train, max_features, col_name):
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                stop_words = None, max_features=max_features)
    
    sentences = labeled_train[col_name].apply(lambda x:" ".join(x))
    train_data_features = vectorizer.fit_transform(sentences).toarray()

    col = ["bow_%s_%s" % (col_name, data) for data in vectorizer.get_feature_names()]
    #col = ["bow%s%s" % (col_name, str(i)) for  in range(0, max_features)]
    df_bow = pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)
    
    labeled_train = pd.concat([labeled_train, df_bow],axis=1)
    
    return labeled_train

##LDA

In [4]:
import gensim
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

def make_lda(train, keep_n, num_topics, col_name):

    data = train[col_name]

    dictionary = corpora.Dictionary(data)
    dictionary.filter_extremes(keep_n=keep_n)

    corpus = [dictionary.doc2bow(text) for text in data]

    print("Make Lda..")

#    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1,distributed = True)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1)
    
    num = len(train)    
    df = []
    
    for i in range(0,num):
        if i % 10000 == 0:
            print(i)
        
        temp = [i[1] for i in lda.get_document_topics(corpus[i],minimum_probability=0)]
        #temp = max(lda.get_document_topics(corpus[i]),key=lambda item:item[1])[0]
        df.append(temp)
    
    col = ["lda_%s_%d" % (col_name, data) for data in range(0, num_topics)]
    df = pd.DataFrame(df, columns = col)
    df.index = train.index
    
    train = pd.concat([train, df], axis=1)
        
    return train

##Word2vec

In [1]:
from gensim.models import word2vec

def makeFeatureVec(words, model, num_features):
    
    featureVec = np.zeros((num_features,), dtype = "float32")
    
    nwords = 0
    
    index2word_set = set(model.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
    
    if nwords != 0:
        featureVec = np.divide(featureVec, nwords)
    
    return featureVec

def getAvgFeatureVecs(texts, model, num_features):
    
    counter = 0
    
    textFeatureVecs = np.zeros((len(texts), num_features), dtype = "float32")
    
    for i, text in enumerate(texts):
        
        if i % 10000 == 0:
            print(i)
        
        textFeatureVecs[i] = makeFeatureVec(text , model, num_features)
        
    return textFeatureVecs

def make_word2vec(train, col_name):
    num_features = 300
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3
    
    sentences = " ".join(train[col_name].apply(lambda x:" ".join(x)))
    
    model = word2vec.Word2Vec(sentences, workers = num_workers, size = num_features,\
                             min_count = min_word_count, window = context, sample = downsampling)
    
    col = ["word2vec_%s_%d" % (col_name, data) for data in range(0, num_features)]
    
    train_feature = getAvgFeatureVecs(train[col_name].apply(lambda x:" ".join(x)), model, num_features)
    train_feature = pd.DataFrame(train_feature, index = train.index, columns = col)
    
    train = pd.concat([train, train_feature], axis = 1)
    
    return train

In [6]:
max_features = 1000
#%time labeled_train = make_lda(labeled_train, 5000, 1000, "text_pos")
#%time labeled_train = make_bag_of_words(labeled_train, 1000, "text_pos")

%time labeled_train = make_word2vec(labeled_train, "text_pos")

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
CPU times: user 27min 26s, sys: 12min 17s, total: 39min 43s
Wall time: 30min 59s


### predictor, model 생성

In [7]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['text_pos', label])

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

###cross validation

In [8]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")
cv_result = scores.mean()

print(cv_result)

0.63900741504


##Result
  * text의 morphs를 bag of words로 변환 (feature : 1,000개) - 0.637526911627
  * text의 pos를 bag of words로 변환 (feature : 1,000개) - 0.661265083065
  * 데이터를 셔플하고 text의 pos를 bag of words로 변환 (feature : 1,000개) - 0.688875771784
  
  * text의 pos를 lda로 변환 (keep_n : 5,000개, num_topics : 1,000개) - 0.643143549702
  * text의 pos를 word2vec (num_features = 300) - 0.63900741504
  
  * bs의 파라미터를 "lxml"이 아닌 "html.parser"로 주고 pos를 bag of words로 변환 (feature : 1,000개) - 0.657726939833

In [9]:
import pickle

pickle.dump(labeled_train[pre], open("text_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)